In [ ]:
import pandas as pd
import os
import shutil
import re
import datetime

In [ ]:
df = pd.read_csv('ConversionComma.csv')
df[["RN100550326"]]

In [ ]:
os.mkdir(r"Downloads\TestScript\created")

In [ ]:
cleaner_spreadsheet_path = r"Documents\Workflow\Conversion.xlsx"
cleaner_data = pd.read_excel(cleaner_spreadsheet_path)
cleaner_data.dropna(inplace=True)
cleaner_data

In [ ]:
file_names = []

for index, row in cleaner_data.iterrows():
    ending = "DS" if row["TYPE"] == "DROP STATION REGISTRATION" else "Fac"
    file_names.append(f"{index}-DCR_{row["CUTID"]}_{ending}")
file_names

In [ ]:
shutil.copy2(r"Documents\Workflow\data_template.xlsx", r"Documents\Workflow\copy.xlsx")

In [ ]:
subfolder_path = pd.read_csv(r'Documents\Workflow\ori_subfolder_storage.csv', index_col=0)
subfolder_path.iloc[0].iloc[0]

In [ ]:
my_list = ['a', 'b', 'c']
my_series = pd.Series(my_list)
my_series.to_csv("test_series.csv")
my_series = pd.read_csv("test_series.csv")
my_series

In [ ]:
document_data_path = r'Documents\Workflow\document_data.xlsx'
document_data = pd.read_excel(document_data_path)
document_data["TYPE"] = document_data["TYPE"].fillna(0)
document_data["SUS"] = document_data["SUS"].fillna("F")

document_data

In [ ]:
def convert_month(month):
    month = month.lower()
    match month:
        case "jan":
            return "01"
        case "feb":
            return "02"
        case "mar":
            return "03"
        case "apr":
            return "04"
        case "may":
            return "05"
        case "jun":
            return "06"
        case "jul":
            return "07"
        case "aug":
            return "08"
        case "sep":
            return "09"
        case "oct":
            return "10"
        case "nov":
            return "11"
        case "dec":
            return "12"
        case _:
            return "-1"

def reformat_date(date):
    date = date.replace("-", "/")
    #mm/dd/yyyy
    type1 = r"(0?[1-9]|1[0,1,2])\/(0?[1-9]|[12][0-9]|3[01])\/(19|20)\d{2}"
    #mm/dd/yy
    type2 = r"(0?[1-9]|1[0,1,2])\/(0?[1-9]|[12][0-9]|3[01])\/\d{2}"
    #yyyymmdd
    type3 = r"(19|20)\d{2}(0?[1-9]|1[0,1,2])(0[1-9]|[12][0-9]|3[01])"
    #mmm dd yyyy
    type4 = r"([a-zA-Z]{3}) (0?[1-9]|[12][0-9]|3[01]) (19|20)\d{2}"
    #mmm dd yyyy
    type5 = r"([a-zA-Z]{3}) (0?[1-9]|[12][0-9]|3[01]) (19|20)\d{2}"

    regex1 = re.compile(type1)
    regex2 = re.compile(type2)
    regex3 = re.compile(type3)
    regex4 = re.compile(type4)
    regex5 = re.compile(type5)

    if(regex1.match(date)):
        date_obj = datetime.datetime.strptime(date, "%m/%d/%Y")
        return date_obj.strftime("%Y%m%d")
    elif(regex2.match(date)):
        year_digits = date[len(date) - 2: len(date)]
        if(int(year_digits) > 50):
            year_digits = "19" + year_digits
        else:
            year_digits = "20" + year_digits

        date = date[:-2]
        date = date + year_digits
        date_obj = datetime.datetime.strptime(date, "%m/%d/%Y")
        return date_obj.strftime("%Y%m%d")
    elif(regex3.match(date)):
        return date
    elif(regex4.match(date)):
        month_val = convert_month(date[0:3])
        if(month_val == "-1"):
            return "-1"
        date_obj = datetime.datetime.strptime(month_val + date[3:], "%m %d %Y")
        return date_obj.strftime("%Y%m%d")
    elif(regex5.match(date)):
        
        year_digits = date[len(date) - 2: len(date)]
        if(int(year_digits) > 50):
            year_digits = "19" + year_digits
        else:
            year_digits = "20" + year_digits

        date = date[:-2]
        date = date + year_digits

        month_val = convert_month(date[0:3])
        if(month_val == "-1"):
            return "-1"
        date_obj = datetime.datetime.strptime(month_val + date[3:], "%m %d %Y")
        return date_obj.strftime("%Y%m%d")
    else:
        return "-1"
    
result = []

# for date in document_data["DATE"]:
#     result.append(str(date) + "--" + str(reformat_date(str(date))))

result = reformat_date("2/8/12")

result

In [ ]:
def extract_path_details(path):
    end = path.rfind("\\")
    start = 44

    path = path[start:end]
    dcr_start = path.find("DCR")
    dcr_end = path.rfind("_")
    dcr = path[dcr_start: dcr_end]
    location_type = path[dcr_end:]
    location_type = location_type.replace("_","-")
    return (dcr, location_type)

path_det = extract_path_details(r"Documents\Workflow\Result\0-DCR_15524_DS\mod_simplex")
path_det